In [5]:
import glob
from codecs import open

import pandas as pd
import numpy as np
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/emersonsjsu/GitHub/EarlGrey/google_private_key.json
WORKING_DIRECTORY = "/Users/emersonsjsu/GitHub/EarlGrey"

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/emersonsjsu/GitHub/EarlGrey/google_private_key.json


In [6]:
my_index = pd.MultiIndex(levels=[[], []],
                         labels=[[], []],
                         names=[u'character_name', u'episode_no'])
my_columns = [u'text',u'sentiment_score',u'sentiment_magnitude']
char_df = pd.DataFrame(index=my_index, columns=my_columns)
char_df

,,text,sentiment_score,sentiment_magnitude
character_name,episode_no,,,


In [7]:
# Clear char_df
char_df = char_df.iloc[0:0]
FIRST_EPISODE_OFFSET = 101


def sanitize_name(name_line):
    name = name_line.strip()  # Voice overs count as same character
    if name.endswith('V.O.'):
        name = name[:-5]
    # Strip parentheticals from name
    if name.find('(') != -1:
        name = name[:name.find('(')]
    # Strip 'S VOICE from name
    if name.endswith('\'S VOICE'):
        name = name[:-8]
    # Strip 'S COM VOICE from name
    if name.endswith('\'S COM VOICE'):
        name = name[:-12]
    return name.strip()


# for file_path in glob.glob(WORKING_DIRECTORY+'/scripts/225.txt'):  # Only read one file for debug purposes
for file_path in glob.glob(WORKING_DIRECTORY+'/scripts/*.txt'):
    current_file = open(file_path, "r", encoding='utf-8', errors="ignore")
    episode_no = int(current_file.name[current_file.name.rfind('/') + 1:-4]) - FIRST_EPISODE_OFFSET
    lines = current_file.readlines()
    character_name = ''
    for line in lines:
        # Character dialogue has ended, reset current character
        if line.strip() == '':
            character_name = ''
        # All names are preceded by exactly 5 tabs
        elif line[:5] == '\t\t\t\t\t' and line[5:6] != '\t' and not line.strip().startswith('('):
            # We have found a name! Update current name
            character_name = sanitize_name(line)
        # If we are still looking at a character's dialogue
        elif character_name != '':
            line_text = line.strip()
            # Previous dialogue found for character this episode, append line to previous text
            if (character_name, episode_no) in char_df.index:
                char_df.loc[(character_name, episode_no), 'text'] += ' ' + line_text
            # First dialogue for character this episode, initialize string list
            else:
                char_df.loc[(character_name, episode_no), 'text'] = line_text
char_df.loc[('PICARD',124),'text']


/Users/emersonsjsu/.virtualenvs/earlgrey/lib/python2.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


u'Captain\'s log, stardate 45944.1. We\'re en route to Starbase Two-eighteen after a magnetic wave survey of the Parvenium Sector. We will give our report directly to Fleet Admiral Gustafson. The last time I encountered Admiral Gustafson... I ended up spending nine straight hours at the opera. The entire "Ring" cycle at one sitting... And for me... but not, apparently, for Admiral Gustafson. She went back the next day and sat through it all again. I warned her that this time -- On screen. Magnify. Mister Data? Increase power to -- Where is this... ? Computer, freeze program. (beat) End program. Picard to Enterprise -- I asked you... what is this place? Where is my uniform? My communicator? I am Captain Jean-Luc Picard of the Federation Starship Enterprise. I would like to speak with whoever is in charge here. Just tell me this -- am I a prisoner here? Are you in charge here? I want to be returned to my ship immediately. Please... if you could tell me... what is this place? Where am I? 

In [19]:
def analyze(text):
    client = language.LanguageServiceClient()
    document = types.Document(
        language='EN',
        content=text,
        type=enums.Document.Type.PLAIN_TEXT
    )
    annotations = client.analyze_sentiment(document=document)
    for index, sentence in enumerate(annotations.sentences):
        sentence_sentiment = sentence.sentiment.score
        print('Sentence {} has a sentiment score of {}'.format(
            index, sentence_sentiment))

    print('Overall Sentiment: score of {} with magnitude of {}'.format(
        annotations.document_sentiment.score, annotations.document_sentiment.magnitude))
    return annotations.document_sentiment.score, annotations.document_sentiment.magnitude


char_df['sentiment_score'], char_df['sentiment_magnitude'] = np.vectorize(analyze)(char_df.loc[('PICARD',124),'text'])


Sentence 0 has a sentiment score of 0.0
Sentence 1 has a sentiment score of 0.20000000298
Sentence 2 has a sentiment score of 0.40000000596
Sentence 3 has a sentiment score of 0.10000000149
Sentence 4 has a sentiment score of -0.20000000298
Sentence 5 has a sentiment score of 0.0
Sentence 6 has a sentiment score of -0.10000000149
Sentence 7 has a sentiment score of 0.0
Sentence 8 has a sentiment score of -0.20000000298
Sentence 9 has a sentiment score of 0.10000000149
Sentence 10 has a sentiment score of -0.10000000149
Sentence 11 has a sentiment score of -0.10000000149
Sentence 12 has a sentiment score of -0.699999988079
Sentence 13 has a sentiment score of 0.20000000298
Sentence 14 has a sentiment score of -0.20000000298
Sentence 15 has a sentiment score of -0.5
Sentence 16 has a sentiment score of 0.20000000298
Sentence 17 has a sentiment score of 0.0
Sentence 18 has a sentiment score of 0.20000000298
Sentence 19 has a sentiment score of -0.10000000149
Sentence 20 has a sentiment sc

Sentence 0 has a sentiment score of 0.0
Sentence 1 has a sentiment score of 0.20000000298
Sentence 2 has a sentiment score of 0.40000000596
Sentence 3 has a sentiment score of 0.10000000149
Sentence 4 has a sentiment score of -0.20000000298
Sentence 5 has a sentiment score of 0.0
Sentence 6 has a sentiment score of -0.10000000149
Sentence 7 has a sentiment score of 0.0
Sentence 8 has a sentiment score of -0.20000000298
Sentence 9 has a sentiment score of 0.10000000149
Sentence 10 has a sentiment score of -0.10000000149
Sentence 11 has a sentiment score of -0.10000000149
Sentence 12 has a sentiment score of -0.699999988079
Sentence 13 has a sentiment score of 0.20000000298
Sentence 14 has a sentiment score of -0.20000000298
Sentence 15 has a sentiment score of -0.5
Sentence 16 has a sentiment score of 0.20000000298
Sentence 17 has a sentiment score of 0.0
Sentence 18 has a sentiment score of 0.20000000298
Sentence 19 has a sentiment score of -0.10000000149
Sentence 20 has a sentiment sc

In [181]:
char_df.to_pickle(WORKING_DIRECTORY+'/char_df.pickle')

In [3]:
temp = pd.read_pickle(WORKING_DIRECTORY+'/char_df.pickle')


In [4]:
temp

,,text,sentiment_score,sentiment_magnitude
character_name,episode_no,,,
PICARD,161,"Captain's Log, Stardate 47410.2. The Atrean go...",0.0,5.100000
JULIANA,161,"Captain, our situation has worsened since my h...",0.0,51.900002
PRAN,161,Our gravitational field has been affected -- s...,0.1,6.900000
GEORDI,161,We could minimize seismic activity by creating...,0.0,8.100000
DATA,161,The only permanent solution would be to re-liq...,0.0,48.299999
TROI,161,"But... ? She's your mother, Data. That's how p...",0.0,4.000000
RIKER,161,Bridge to Engineering. We've established synch...,0.0,5.600000
WORF,161,Phasers are locked on target. Standing by. (wo...,0.0,0.200000
BEVERLY,161,(smiles) I see where Data gets his musical tal...,0.0,5.000000


In [ ]:
temp.head()